<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## ***DATA SCIENCE / SECTION 4 / SPRINT 2 / NOTE 2***

---

# 단어를 분산 표현(Distributed Representation)으로 나타내기

## 🏆 학습 목표

- **단어의 분산 표현(Distributed Representation)**
    - 원-핫 인코딩의 개념과 단점에 대해서 이해할 수 있습니다.
    - 분포 기반의 표현, 임베딩이 무엇인지 설명할 수 있습니다. 

- **Word2Vec**
    - CBoW와 Skip-gram의 차이에 대해서 설명할 수 있습니다.
    - Word2Vec의 임베딩 벡터를 시각화한 결과가 어떤 특징을 가지는지 설명할 수 있습니다.

- **fastText**
    - OOV 문제가 무엇인지에 대해 설명할 수 있습니다.
    - 철자(Character) 단위 임베딩 방법의 장점에 대해 설명할 수 있습니다.

## 🛫 Warm up

- 다음 영상을 시청하세요.
    - [Word2Vec](https://youtu.be/sY4YyacSsLc)

## 1. Distributed Representation

이전 시간에 문서 내에 단어가 등장하는 횟수를 기반으로 문서를 벡터화하는 **등장 횟수 기반 표현(Count-based Representation)**에 대해서 알아보았습니다.



이번 시간에는 단어 자체를 벡터화하는 방법에 대해서 알아보고자 합니다.<br/>
이번 시간에 배우게 될 Word2Vec, fastText는 우리가 벡터로 표현하고자 하는 타겟 단어(Target word)가 해당 단어 주변 단어에 의해 결정됩니다.

단어 벡터를 이렇게 정하는 이유는 [분포 가설(Distribution hypothesis)](https://en.wikipedia.org/wiki/Distributional_semantics) 때문입니다.<br/>
분포 가설은 다음과 같습니다.

> **'비슷한 위치에서 등장하는 단어들은 비슷한 의미를 가진다'**

<img src="http://cdn.ppomppu.co.kr/zboard/data3/2018/1004/m_1538578985_9861_3dba059e_23ec_43dc_9f27_ce1d58c00e71.png.jpg">

'유유상종'이라는 사자성어 처럼 <br/>
**비슷한 의미를 지닌 단어는 주변 단어 분포도 비슷하다**는 것이지요.

예를 들어, 두 문장

- I found **good** stores.
- I found **beautiful** stores.

에서 _"**good** 과 **beautiful** 은 해당 단어 주변에 분포한 단어가 유사하기 때문에 비슷한 의미를 지닐 것이다"_ 라고 가정하는 것이 분포 가설입니다.

바로 이 분포 가설에 기반하여 주변 단어 분포를 기준으로 단어의 벡터 표현이 결정되기 때문에 **<font color="ff6f61">분산 표현(Distributed representation)</font>**이라고 부르게 됩니다.

본격적으로 분산 표현을 배우기에 앞서, **원-핫 인코딩**에 대해 알아보겠습니다.

### 1) 원-핫 인코딩(One-hot Encoding)

원-핫 인코딩은 단어를 벡터화하고자 할 때 선택할 수 있는 가장 쉬운 방법입니다. <br/>
표 형태로 된 데이터를 다룰 때 범주형 변수를 요소마다 [0 0 0 1 ...] 의 형태로 나타냈던 것과 동일한 방법을 사용합니다.

> "I am a student"

라는 문장에서 각 단어를 원-핫 인코딩으로 나타내면 다음과 같습니다.

> I : [1 0 0 0] <br/>
> am : [0 1 0 0] <br/>
> a : [0 0 1 0] <br/>
> student : [0 0 0 1]

In [ ]:
sent = "I am a student"
word_lst = sent.split()

word_dict = {}

for idx, word in enumerate(word_lst):
    vec = [0 for _ in range(len(word_lst))]
    vec[idx] = 1

    word_dict[word] = vec

print(word_dict)

{'I': [1, 0, 0, 0], 'am': [0, 1, 0, 0], 'a': [0, 0, 1, 0], 'student': [0, 0, 0, 1]}


쉽게 이해할 수 있는 직관적인 방법이지만 원-핫 인코딩에는 치명적인 단점이 있습니다.<br/>
바로 단어 간 유사도를 구할 수 없다는 점입니다.

단어 간 유사도를 구할 때에는 코사인 유사도(cosine similarity)가 자주 사용됩니다.<br/>
코사인 유사도를 구하기 위한 식은 다음과 같습니다.

$$
\large \text{Cosine similarity} = \frac{\vec{a} \cdot \vec{b} }{\vert \vec{a} \vert \vert \vec{b} \vert }
$$

원-핫 인코딩을 사용한 두 벡터의 내적은 항상 0이므로 어떤 두 단어를 골라 코사인 유사도를 구하더라도 그 값은 0이 됩니다.<br/>
이렇게 두 단어 사이의 관계를 전혀 알 수 없다는 것이 원-핫 인코딩의 최대 단점입니다.

In [ ]:
import numpy as np

def cos_sim(a, b):
    arr_a = np.array(a)
    arr_b = np.array(b)

    result = np.dot(arr_a, arr_b)/(np.linalg.norm(arr_a)*np.linalg.norm(arr_b))
    return result

print(f"I 와 am 의 코사인 유사도 : {cos_sim(word_dict['I'], word_dict['am'])}")
print(f"I 와 student 의 코사인 유사도 : {cos_sim(word_dict['I'], word_dict['student'])}")

I 와 am 의 코사인 유사도 : 0.0
I 와 student 의 코사인 유사도 : 0.0


### 2) 임베딩(Embedding)

원-핫 인코딩의 단점을 해결하기 위해 등장한 것이 **<font color="ff6f61">임베딩(Embedding)</font>**입니다. <br/>
단어를 고정 길이의 벡터, 즉 차원이 일정한 벡터로 나타내기 때문에 '임베딩'이라는 이름이 붙었습니다.<br/>
임베딩을 거친 단어 벡터는 원-핫 인코딩과는 다른 형태의 값을 가집니다.

예를 들어,

> [0.04227, -0.0033, 0.1607, -0.0236, ...]

위와 같이 벡터 내의 각 요소가 연속적인 값을 가지게 됩니다.<br/>
어떻게 이런 벡터가 만들어지는지, 가장 널리 알려진 임베딩 방법인 **`Word2Vec`** 를 통해 알아보도록 하겠습니다.

> ❗️ ***임베딩은 자연어처리 이외에 다른 딥러닝 분야(추천 시스템, GNN 등)에서도 사용되니 잘 기억해두도록 합시다. :)***

## 2. Word2Vec

2013년에 고안된 **<font color="ff6f61">Word2Vec</font>** 은 말 그대로 **단어를 벡터로(Word to Vector) 나타내는 방법**으로 가장 널리 사용되는 임베딩 방법 중 하나입니다.<br>
**`Word2Vec`**은 특정 단어 양 옆에 있는 두 단어(window size = 2)의 관계를 활용하기 때문에 **분포 가설을 잘 반영**하고 있습니다.

`Word2Vec` 에는 `CBoW`와 `Skip-gram`의 2가지 방법이 있습니다.<br/>
두 가지 방법이 어떻게 다른지에 대해서 알아보도록 하겠습니다.

### 1) CBoW 와 Skip-gram

`CBoW`와 `Skip-gram`의 차이는 

1. 주변 단어에 대한 정보를 기반으로 중심 단어의 정보를 예측하는 모델인지 ▶️ **<font color="ff6f61">CBoW(Continuous Bag-of-Words)</font>**
2. 중심 단어의 정보를 기반으로 주변 단어의 정보를 예측하는 모델인지 ▶️ **<font color="ff6f61">Skip-gram</font>**

에 따라서 달라집니다.

아래 그림을 통해 두 방식의 차이를 좀 더 잘 이해해보도록 하겠습니다.

<img src="https://www.researchgate.net/profile/Nailah_Al-Madi/publication/319954363/figure/fig1/AS:552189871353858@1508663732919/CBOW-and-Skip-gram-models-architecture-1.png" width="800" />

예시를 통해서도 둘의 차이를 알아보겠습니다.<br/>
<별 헤는 밤> 의 일부분에 형태소 분석기를 적용하여 토큰화한 것입니다.

> “… 어머님 나 는 별 하나 에 아름다운 말 한마디 씩 불러 봅니다 …”

**CBoW** 를 사용하면 표시된 단어 정보를 바탕으로 아래의 [ ---- ] 에 들어갈 단어를 예측하는 과정으로 학습이 진행됩니다.

> “… 나 는 [ -- ] 하나 에 … “ <br/>
> “… 는 별 [ ---- ] 에 아름다운 …”<br/>
> “… 별 하나 [ -- ] 아름다운 말 …”<br/>
> “… 하나 에 [ -------- ] 말 한마디 …”

**Skip-gram** 을 사용하면 표시된 단어 정보를 바탕으로 다음의 [ ---- ] 에 들어갈 단어를 예측하는 과정으로 학습이 진행됩니다.

> “… [ -- ] [ -- ] 별 [ ---- ] [ -- ] …” <br/>
> “… [ -- ] [ -- ] 하나 [ -- ] [ -------- ] …” <br/>
> “… [ -- ] [ ---- ] 에 [ -------- ] [ -- ] …” <br/>
> “… [ ---- ] [ -- ] 아름다운 [ -- ] [ ------ ] …”

더 많은 정보를 바탕으로 특정 단어를 예측하기 때문에 CBoW의 성능이 더 좋을 것으로 생각하기 쉽지만,<br/>
역전파 관점에서 보면 Skip-gram에서 훨씬 더 많은 학습이 일어나기 때문에 **Skip-gram의 성능이 조금 더 좋게 나타납니다.**<br/>
물론 계산량이 많기 때문에 Skip-gram에 드는 리소스가 더 큰 것도 사실입니다.

### Word2Vec 모델의 구조

성능 덕분에 조금 더 자주 사용되는 Skip-gram을 기준으로 Word2Vec의 구조에 대해 알아보겠습니다.

- 입력 : Word2Vec의 입력은 원-핫 인코딩된 단어 벡터입니다.
- 은닉층 : 임베딩 벡터의 차원수 만큼의 노드로 구성된 은닉층이 1개인 신경망입니다.
- 출력층 : 단어 개수 만큼의 노드로 이루어져 있으며 활성화 함수로 소프트맥스를 사용합니다.

아래 그림을 통해서 Word2Vec 모델의 개략적인 구조에 대해 알아보겠습니다.

논문에서는 총 10,000개의 단어에 대해서 300차원의 임베딩 벡터를 구했기 때문에<br/>
신경망 구조가 아래와 같아졌습니다.

<img src="http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png" width="800" />


### Word2Vec 학습을 위한 학습 데이터 디자인

효율적인 Word2Vec 학습을 위해서는 학습 데이터를 잘 구성해야 합니다.<br/>
Window 사이즈가 2인 Word2Vec 이므로 중심 단어 옆에 있는 2개 단어에 대해 단어쌍을 구성합니다.

예를 들어, **"The tortoise jumped into the lake"** 라는 문장에 대해 단어쌍을 구성해보겠습니다.<br/>
윈도우 크기가 2인 경우 다음과 같이 Skip-gram을 학습하기 위한 데이터 쌍을 구축할 수 있습니다.

- 중심 단어 : **The**, 주변 문맥 단어 : tortoise, jumped
    - 학습 샘플: (the, tortoise), (the, jumped)
- 중심 단어 : **tortoise**, 주변 문맥 단어 : the, jumped, into
    - 학습 샘플: (tortoise, the), (tortoise, jumped), (tortoise, into)
- 중심 단어 : **jumped**, 주변 문맥 단어 : the, tortoise, into, the
    - 학습 샘플: (jumped, the), (jumped, tortoise), (jumped, into), (jumped, the)
- 중심 단어 : **into**, 주변 문맥 단어 : tortoise, jumped, the, lake
    - 학습 샘플: (into, tortoise), (into, jumped), (into, the), (into, lake)

이런 방법으로 학습 데이터를 만들면 다음과 같은 데이터쌍이 만들어 집니다. 

|중심단어|문맥단어|
|---------|---------|
|the|tortoise|
|the|jumped|
|tortoise|the|
|tortoise|jumped|
|tortoise|into|
|jumped|the|
|jumped|tortoise|
|jumped|into|
|jumped|the|
|into|tortoise|
|into|jumped|
|into|the|
|into|lake|
|...|...|

Skip-gram 에서는 중심단어를 입력으로, 문맥단어를 레이블로 하는 분류(Classification)를 통해 학습한다고 생각하시면 되겠습니다.

### Word2Vec의 결과

학습이 모두 끝나면 10000개의 단어에 대해 300차원의 임베딩 벡터가 생성됩니다. <br/>
만약에 임베딩 벡터의 차원을 조절하고 싶다면 은닉층의 노드 수를 줄이거나 늘릴 수 있습니다.<br/>

아래 그림은 신경망 내부에 있는 $10000 \times 300$ 크기의 가중치 행렬에 의해서<br/>
10000개 단어에 대한 300차원의 벡터가 생성되는 모습을 나타낸 이미지입니다.

<img src="https://i.imgur.com/1ETMljf.png" width="600" />


학습과정에서 효율을 높이기 위해 사용하는 기법들이 있지만 아직은 너무 깊게 들어가지 않겠습니다.<br/>
추후 아래 키워드를 통해 Word2Vec을 조금 더 적은 계산으로 하는 방법에 대해 조사해보시면 좋겠습니다.

- 더 알아보기
    - Sub-sampling
    - Negative-sampling

결과적으로 Skip-gram 모델을 통해 10000개 단어에 대한 임베딩 벡터를 얻을 수 있습니다.<br/>
이렇게 얻은 임베딩 벡터는 문장 간의 관련도 계산, 문서 분류같은 작업에 사용할 수 있습니다.

### Word2Vec으로 임베딩한 벡터 시각화

Word2Vec을 통해 얻은 임베딩 벡터는 **<font color="ff6f61">단어 간의 의미적, 문법적 관계를 잘 나타냅니다.</font>**<br/>
이를 대표적으로 잘 보여주는 것이 아래 그림입니다.<br/>

1. **`man - woman`** 사이의 관계와 **`king - queen`** 사이의 관계가 매우 유사하게 나타납니다.<br/>
생성된 임베딩 벡터가 단어의 **의미적(Semantic) 관계를 잘 표현**하는 것을 확인할 수 있습니다.

2. **`walking - walked`** 사이의 관계와 **`swimming - swam`** 사이의 관계가 매우 유사하게 나타납니다.<br/>
생성된 임베딩 벡터가 단어의 **문법적(혹은 구조적, Syntactic)인 관계도 잘 표현**하는 것을 확인할 수 있습니다.

3. 고유명사에 대해서도 나라 - 수도 와 같은 관계를 잘 나타내고 있는 것을 확인할 수 있습니다.

<img src="https://miro.medium.com/max/3010/1*OEmWDt4eztOcm5pr2QbxfA.png"/>

### gensim 패키지로 word2Vec 실습하기

**`gensim`** 은 **`Word2Vec`** 으로 사전 학습된 임베딩 벡터를 쉽게 사용해볼 수 있는 패키지입니다.<br/>
gensim을 사용하여 word2vec의 결과가 어떻게 도출되는지 알아보겠습니다.

0. (시작하기 전에) **`gensim` 패키지를 최신 버전으로 업그레이드** 합니다.

아래 `--upgrade` 셀을 실행하여 패키지를 업그레이드 한 후,<br/>
메뉴 탭에서 '런타임' > '런타임 다시 시작'을 클릭하여 런타임을 다시 시작합니다.

이후 아래 `.__version__` 이 있는 셀을 활용하여 최신 버전인지 확인합니다.

In [ ]:
!pip install gensim --upgrade

     |████████████████████████████████| 23.9MB 160kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gensim

gensim.__version__

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


'4.0.1'

1. **구글 뉴스 말뭉치로 학습된 `word2vec` 벡터를 다운받습니다.** (시간이 오래 걸립니다.)

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


2. **0 ~ 9 인덱스에 위치한 단어가 무엇인지 확인해봅시다.**

In [ ]:
for idx, word in enumerate(wv.index_to_key):
    if idx == 10:
        break

    print(f"word #{idx}/{len(wv.index_to_key)} is '{word}'")

word #0/3000000 is '</s>'
word #1/3000000 is 'in'
word #2/3000000 is 'for'
word #3/3000000 is 'that'
word #4/3000000 is 'is'
word #5/3000000 is 'on'
word #6/3000000 is '##'
word #7/3000000 is 'The'
word #8/3000000 is 'with'
word #9/3000000 is 'said'


3. **임베딩 벡터의 차원과 값을 눈으로 확인해봅시다.**

**king** 이라는 단어의 벡터의 shape을 출력하여 임베딩 벡터의 차원을 확인해봅시다.<br/>
결과를 통해 **`Word2Vec`** 을 통해 학습된 임베딩 벡터는 300차원이며, 벡터의 요소가 원-핫 인코딩과는 다르다는 것을 확인할 수 있습니다.

In [ ]:
vec_king = wv['king']

print(f"Embedding dimesion is : {vec_king.shape}\n")
print(f"Embedding vector of 'king' is \n\n {vec_king}")

Embedding dimesion is : (300,)

Embedding vector of 'king' is 

 [ 1.25976562e-01  2.97851562e-02  8.60595703e-03  1.39648438e-01
 -2.56347656e-02 -3.61328125e-02  1.11816406e-01 -1.98242188e-01
  5.12695312e-02  3.63281250e-01 -2.42187500e-01 -3.02734375e-01
 -1.77734375e-01 -2.49023438e-02 -1.67968750e-01 -1.69921875e-01
  3.46679688e-02  5.21850586e-03  4.63867188e-02  1.28906250e-01
  1.36718750e-01  1.12792969e-01  5.95703125e-02  1.36718750e-01
  1.01074219e-01 -1.76757812e-01 -2.51953125e-01  5.98144531e-02
  3.41796875e-01 -3.11279297e-02  1.04492188e-01  6.17675781e-02
  1.24511719e-01  4.00390625e-01 -3.22265625e-01  8.39843750e-02
  3.90625000e-02  5.85937500e-03  7.03125000e-02  1.72851562e-01
  1.38671875e-01 -2.31445312e-01  2.83203125e-01  1.42578125e-01
  3.41796875e-01 -2.39257812e-02 -1.09863281e-01  3.32031250e-02
 -5.46875000e-02  1.53198242e-02 -1.62109375e-01  1.58203125e-01
 -2.59765625e-01  2.01416016e-02 -1.63085938e-01  1.35803223e-03
 -1.44531250e-01 -5.68847

4. **말뭉치에 등장하지 않는 단어의 임베딩 벡터를 확인해봅시다.**

**cameroon** 이라는 단어는 구글 뉴스 말뭉치에 등장하지 않는 단어(Unknown token)입니다.<br/>
이 단어를 위와 같이 임베딩 벡터화 해보겠습니다.<br/>
아래 결과에서는 **`KeyError`** 가 발생합니다.<br/>
이처럼 **`Word2Vec`**은 말뭉치에 등장하지 않는 단어는 벡터화 할 수 없다는 단점이 있습니다.

In [ ]:
unk = 'cameroon'

try:
    vec_unk = wv[unk]
except KeyError:
    print(f"The word #{unk} does not appear in this model")

The word #cameroon does not appear in this model


5. **단어 간 유사도를 파악해봅시다.**

**`gensim`** 패키지가 제공하는 **`.similarity`** 를 활용하면 단어 간 유사도를 파악할 수 있습니다.<br/>
원-핫 인코딩과 다르게 임베딩 벡터는 단어 간 유사도가 0이 아닌 값으로 나오게 됩니다.<br/>
아래는 **'car'** 와 몇몇 단어의 유사도를 비교한 결과입니다.

In [ ]:
pairs = [
    ('car', 'minivan'),   
    ('car', 'bicycle'),  
    ('car', 'airplane'),
    ('car', 'cereal'),    
    ('car', 'democracy')
]

for w1, w2 in pairs:
    print(f'{w1} ======= {w2}\t  {wv.similarity(w1, w2):.2f}')

car ======= minivan	  0.69
car ======= bicycle	  0.54
car ======= airplane	  0.42
car ======= cereal	  0.14
car ======= democracy	  0.08


`.most_similar` 메서드를 사용하여

`'car'`벡터에 `'minivan'` 벡터를 더한 벡터와 가장 유사한 5개의 단어를 뽑아보겠습니다.

In [ ]:
for i, (word, similarity) in enumerate(wv.most_similar(positive=['car', 'minivan'], topn=5)):
    print(f"Top {i+1} : {word}, {similarity}")

Top 1 : SUV, 0.8532192707061768
Top 2 : vehicle, 0.8175783753395081
Top 3 : pickup_truck, 0.7763688564300537
Top 4 : Jeep, 0.7567334175109863
Top 5 : Ford_Explorer, 0.7565720081329346


시각화에서 확인한 것처럼 `king` 벡터에 `women` 벡터를 더한 뒤 `men` 벡터를 빼주면 `queen` 이 나오는 것과<br/>
`walking` 벡터에 `swam` 벡터를 더한 뒤 `walked` 벡터를 빼주면 `swimming` 이 나오는 것을 확인할 수 있습니다.

In [ ]:
print(wv.most_similar(positive=['king', 'women'], negative=['men'], topn=1))
print(wv.most_similar(positive=['walking', 'swam'], negative=['walked'], topn=1))

[('queen', 0.6525818109512329)]
[('swimming', 0.7448815703392029)]


`.doesnt_match` 메서드를 사용하여

['fire', 'water', 'land', 'sea', 'air', 'car'] 중에서 가장 관계 없는 단어를 뽑아봅시다.

In [ ]:
print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

car


## 3. fastText

**`fastText`** 는 **`Word2Vec`** 방식에 철자(Character) 기반의 임베딩 방식을 더해준 새로운 임베딩 방식입니다. <br/>
**fastText** 가 고안된 이유는 무엇일까요?



### 1) OOV(Out of Vocabulary) 문제

데이터 수집에 제법 노력을 쏟더라도 세상 모든 단어가 들어있는 말뭉치를 구하는 것은 불가능합니다.<br/>
같은 어근을 지닌 단어라도 "go, went, gone, goes..." 등 수많은 변형이 있습니다.<br/>
게다가 이전에는 쓰지 않았던 신조어가 등장하기도 합니다.

하지만 **`Word2Vec`** 은 말뭉치에 등장하지 않은 단어에 대해서는 임베딩 벡터를 만들지 못한다는 단점이 있습니다.<br/>
위에서 살펴본 것처럼 말뭉치에 등장하지 않은 단어인 Cameroon의 임베딩 벡터를 출력하려다 보니 에러가 발생했습니다.<br/>
이렇게 기존 말뭉치에 등장하지 않는 단어가 등장하는 문제를 **<font color="ff6f61">OOV(Out of Vocabulary) 문제</font>**라고 합니다.<br/>
Word2Vec은 등장하지 않는 단어에 대해서는 학습하지 않기 때문에 예측(혹은 추론) 단계에서<br/>
Cameroom과 같은 새로운 단어가 등장하면 에러를 발생시킵니다.

또한 적게 등장하는 단어(Rare words)에 대해서는 학습이 적게 일어나기 때문에 적절한 임베딩 벡터를 생성해내지 못한다는 것도 Word2Vec 의 단점입니다.

### 2) 철자 단위 임베딩(Character level Embedding)

- **철자 단위 임베딩이란?**

**`fastText`** 는 철자(Character) 수준의 임베딩을 보조 정보로 사용함으로써 OOV 문제를  해결해냈습니다.

예시를 생각해 보겠습니다.<br/>
"맞벌이"라는 단어를 모른다고 하더라도 아래 단어를 알면 대략적인 의미를 유추해 볼 수 있습니다.

> 1. _"맞선, 맞절, 맞대다, 맞들다, 맞바꾸다, 맞서다, 맞잡다, 맞장구치다"_ <br/>
> 2. _"벌다, 벌어, 벌고"_<br/>
> 3. _"먹이, 깊이, 넓이"_

첫 번째 줄 단어를 통해서 **"맞-"**이라는 접두사의 의미를 유추해 보고,<br/>
두 번째 줄 단어를 통해서 **"-벌-"**이라는 어근의 의미를 유추해 보고,<br/>
세 번째 줄 단어를 통해서 **"-이"**라는 접미사의 의미를 유추해 볼 수 있습니다.<br/>
세 가지를 잘 조합하면 "맞벌이" 라는 단어의 뜻을 알 수 있습니다.

**`fastText`** 가 철자 단위 임베딩을 사용하고자 하는 이유도 동일합니다.<br/>
모델이 학습하지 못한 단어더라도 잘 쪼개고 보면 말뭉치에서 등장했던 단어를 통해 유추해 볼 수 있다는 아이디어에서 출발하였습니다.

- **fastText가 Character-level(철자 단위) 임베딩을 적용하는 법 : <font color="ff6f61">Character n-gram</font>**

**`fastText`** 는 3-6개로 묶은 Character 정보(3-6 grams) 단위를 사용합니다.<br/>
3-6개 단위로 묶기 이전에 모델이 접두사와 접미사를 인식할 수 있도록 해당 단어 앞뒤로 "<", ">" 를 붙여줍니다.<br/>
그리고 나서 해당 단어를 3-6개 Character-level로 잘라서 임베딩을 적용합니다.

> 만약 eating 이라는 단어에 Character-level 임베딩을 적용한다면 3-gram은 다음과 같이 될 것입니다.

<img src="https://amitness.com/images/fasttext-3-gram-sliding.gif" width="300" />

> 이 과정을 거치면 3-gram 에 대해서는 아래와 같은 6개의 철자 단위가 나오게 됩니다.

<img src="https://amitness.com/images/fasttext-3-grams-list.png" width="800" />

이와 같은 방식을 3개 부터 6개까지 진행한 뒤 임베딩 벡터를 생성하고 원래 eating 의 임베딩 벡터와 함께 사용합니다.

| word   | Length(n) | Character n-grams            |
|--------|-----------|------------------------------|
| eating | 3         | <ea, eat, ati, tin, ing, ng> |
| eating | 4         | <eat, eati, atin, ting, ing> |
| eating | 5         | <eati, eatin, ating, ting>   |
| eating | 6         | <eatin, eating, ating>       |

총 18개의 Character-level n-gram을 얻을 수 있습니다.<br/>
fastText에서는 이렇게 얻어진 n-gram 들의 임베딩 벡터를 모두 구하게 됩니다.

꽤 많은 경우의 수를 다루기 때문에 "Word2Vec에 비해 엄청나게 많은 시간이 걸리는 것 아닌가?"라고 생각해볼 수도 있지만,<br/>
fastText의 알고리즘이 매우 효율적으로 구성되어 있기 때문에 시간상으로 엄청난 차이가 나지는 않습니다.

- **철자 단위 임베딩 적용하기**

**eating** 이라는 단어가 말뭉치 내에 있다면 skip-gram으로부터 학습한 임베딩 벡터에 위에서 얻은 18개 Character-level n-gram 들의 벡터를 더해줍니다.<br/>
반대로, **eating** 이라는 단어가 말뭉치에 없다면 18개 Character-level n-gram 들의 벡터만으로 구성합니다. 


### 3) 철자 단위 임베딩 시각화

- **fastText 철자 단위 임베딩 시각화**

fastText의 철자 단위 임베딩이 어떤 관계를 맺고 있는지에 대해 이미지를 통해 알아보겠습니다.

아래에 있는 그림은 X,Y축에 있는 단어 내 character n-gram 에 대하여 서로의 연관관계를 나타낸 그래프입니다.<br/>
빨간색을 나타낼 수록 두 단어 부분 사이에 유사한 관계가 있음을 나타냅니다.


![fasttext1](https://i.imgur.com/nltvwmg.png)

위 그래프에서는 **"ity>"** 와 **"ness>"** 가 상당히 유사한 관계를 보이는 것을 확인할 수 있습니다.<br/>
실제로 둘은 모두 명사를 나타내기 위한 접미사입니다.<br/>
fastText를 이러한 단어의 문법적 구조를 잘 나타낸다는 특징을 가지고 있습니다.

![fasttext2](https://i.imgur.com/YJCm6yP.png)

위 그래프에서는 **"link"** 와 **"nnect, onnect, connec"** 등이 상당히 유사한 관계에 있음을 확인할 수 있습니다.<br/>
3-6개 까지의 연속된 character를 다루고 있기 때문에 connect라는 단어 자체가 포함되지는 않았지만<br/>
connect와 link가 가지고 있는 "연결하다"라는 의미를 n-gram 임베딩 벡터도 유사하게 가지고 있음을 확인할 수 있습니다.

예를 들어, `"connectivity"` 와 `"linkage"`라는 단어의 유사도를 구한다고 해보겠습니다.<br/>
**`Word2Vec`** 은 두 단어 중 하나라도 말뭉치 내에 없다면 에러를 발생시키지만,<br/>
**`fastText`** 는 꽤 높은 정확도로 두 단어의 임베딩 벡터를 구하고 유사도를 나타낼 수 있다는 장점이 있습니다.

이제 코드를 통해 Word2Vec과 fastText의 차이점을 알아보도록 하겠습니다.

### 4) gensim 패키지로 fastText 실습하기

**`gensim`** 를 사용하면 **`fastText`** 도 유추해 볼 수 있습니다.<br/>
gensim을 사용하여 fastText의 결과가 어떻게 도출되는지 알아보겠습니다.

In [ ]:
from pprint import pprint as print
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath

# Set file names for train and test data
corpus_file = datapath('lee_background.cor')

model = FastText(vector_size=100)

# build the vocabulary
model.build_vocab(corpus_file=corpus_file)

# train the model
model.train(
    corpus_file=corpus_file, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words,
)

print(model)

1. **'night' 라는 단어와 'nights'라는 단어가 각각 사전에 있는 지를 확인**해봅시다.

In [ ]:
ft = model.wv
print(ft)

#
# FastText models support vector lookups for out-of-vocabulary words by summing up character ngrams belonging to the word.
#
print(f"night => {'night' in ft.key_to_index}")
print(f"nights => {'nights' in ft.key_to_index}")

'night => True'
'nights => False'


'night' 는 말뭉치에 있지만 'nights'는 말뭉치에 없음을 확인할 수 있습니다.

2. **'night'와 'nights'의 임베딩 벡터**를 확인해봅시다.

In [ ]:
print(ft['night'])

array([-1.21940151e-01,  9.35477093e-02, -2.68753201e-01, -9.21401829e-02,
        5.67255244e-02,  3.27864051e-01,  3.91383469e-01,  5.69616437e-01,
        1.93194106e-01, -2.93112427e-01,  6.31607324e-02, -1.48656189e-01,
       -2.79613197e-01,  5.90286553e-01, -3.61445814e-01, -5.47924638e-01,
        1.34900540e-01, -2.14206606e-01, -4.45417851e-01, -5.28838873e-01,
       -4.67930526e-01,  5.05698696e-02, -5.71829677e-01, -1.30317435e-01,
       -1.92824587e-01, -2.69073665e-01, -5.83209455e-01, -1.01116806e-01,
       -2.19189227e-01,  1.81627348e-01, -2.94398159e-01,  2.68787891e-01,
        8.11280549e-01, -2.19889328e-01,  2.07663789e-01,  2.78767705e-01,
        4.31476295e-01, -2.91228201e-02, -3.44264716e-01, -2.94634223e-01,
        5.33911526e-01, -4.73139107e-01,  1.09619908e-01, -3.26181561e-01,
       -5.31040490e-01, -3.78617615e-01,  3.16744496e-04,  1.84120595e-01,
        2.58488834e-01, -2.17340793e-02,  3.49597663e-01, -5.01383007e-01,
        2.72206813e-01, -

In [ ]:
print(ft['nights'])

array([-0.1060066 ,  0.08164798, -0.23207934, -0.07934359,  0.04771318,
        0.28185087,  0.33920357,  0.49367297,  0.16726466, -0.25471696,
        0.05623917, -0.12683469, -0.24250235,  0.50755054, -0.3133934 ,
       -0.47360393,  0.11576562, -0.18443272, -0.38318864, -0.45724452,
       -0.4011069 ,  0.04254517, -0.49356073, -0.11393045, -0.16512693,
       -0.23088636, -0.5021323 , -0.08505005, -0.18894437,  0.15834409,
       -0.25215933,  0.23156057,  0.6985802 , -0.1893784 ,  0.17935547,
        0.24007267,  0.37383503, -0.02515006, -0.29740998, -0.25486094,
        0.45998394, -0.40748954,  0.09404191, -0.2810496 , -0.4591848 ,
       -0.32557708,  0.0031702 ,  0.15951274,  0.22447531, -0.01768246,
        0.30329126, -0.4329943 ,  0.2356085 , -0.37062383, -0.20580977,
       -0.22011346, -0.17842367, -0.10299528,  0.02632287, -0.27278537,
       -0.3024929 , -0.3700565 , -0.2792666 ,  0.30997086, -0.06514771,
        0.61006665,  0.03797323,  0.00971325,  0.32373375,  0.30

3. **두 단어의 유사도**를 확인해봅시다.

In [ ]:
print(ft.similarity("night", "nights"))

0.9999918


4. 사전에 없는 단어인 **`'nights'` 와 가장 비슷한 단어**는 어떤 것이 있는지 알아봅시다.

In [ ]:
print(ft.most_similar("nights"))

[('night', 0.9999917149543762),
 ('rights', 0.9999875426292419),
 ('flights', 0.9999871850013733),
 ('overnight', 0.9999868273735046),
 ('fighters', 0.9999852776527405),
 ('fighting', 0.9999851584434509),
 ('entered', 0.9999849796295166),
 ('fight', 0.999984860420227),
 ('fighter', 0.9999845027923584),
 ('night.', 0.9999843835830688)]


주로 비슷하게 '생긴', 즉 비슷한 character n-gram이 포함된 단어가 많이 속해있는 것을 볼 수 있습니다.

5. **`Word2Vec`** 에서 했던 것과 같이 가장 관련 없는 단어를 뽑아봅시다.

In [ ]:
print(ft.doesnt_match("night noon fight morning".split()))

'noon'


단어의 뜻만 살펴보면 fight이 나와야 할 것 같지만, 뜬금없게도 noon이 등장했습니다.

위 결과들에서 확인할 수 있는 것처럼,<br/>
`fastText` 임베딩 벡터는 단어의 의미보다는 결과 쪽에 조금 더 비중을 두고 있음을 확인할 수 있습니다.

## 4. 임베딩 벡터를 사용하여 문장 분류 수행하기

임베딩 벡터를 사용하여 문장 분류를 수행해 봅시다.

문장 분류를 사용하는 방법 중 가장 간단한 것은 문장에 있는 **단어 벡터를 모두 더한 뒤에 평균내어 구하는 방법**입니다.<br/>
이게 되나 싶을 정도로 간단하지만, 간단한 문제에 대해서는 꽤 좋은 성능을 보여서 baseline 모델로 많이 사용됩니다.

- **`keras`** 및 **`Word2Vec`**을 사용하여 단어 평균으로 문서 분류하기

1. 필요한 모듈을 `import` 해줍니다.

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import imdb

2. Seed를 정해줍니다.

In [ ]:
tf.random.set_seed(42)

3. 데이터셋을 split 해줍니다.

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=20000)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
print(f"Train set shape : {X_train.shape}")
print(f"Test set shape : {X_test.shape}")

Train set shape : (25000,)
Test set shape : (25000,)


4. 데이터셋이 어떻게 생겼는지 눈으로 확인해봅니다.

In [ ]:
X_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,

5. **인덱스로 된 데이터를 텍스트로 변경하는 함수를 구현합니다.**<br/>
첫 번째 데이터를 텍스트로 변경하고 확인해봅시다.

In [2]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    """
    word_index를 받아 text를 sequence 형태로 반환하는 함수입니다.
    """
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [ ]:
decode_review(X_train[0])

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit the

6. **`keras` 의 `tokenizer` 에 텍스트를 학습**시킵니다.

In [ ]:
sentences = [decode_review(idx) for idx in X_train]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

19999


7. **`pad_sequence` 를 통해 패딩 처리해줍니다.**<br/>
`pad_squence`가 무엇이며 이를 해주는 이유에 대해서는 구글링을 통해 스스로 알아보도록 합니다.<br/>
`maxlen` 을 평균보다 조금 더 긴 400 으로 설정합니다.

In [ ]:
X_encoded = tokenizer.texts_to_sequences(sentences)

#
max_len = max(len(sent) for sent in X_encoded)
print(max_len)

2494


In [ ]:
print(f'Mean length of train set: {np.mean([len(sent) for sent in X_train], dtype=int)}')

Mean length of train set: 238


In [ ]:
X_train=pad_sequences(X_encoded, maxlen=400, padding='post')
y_train=np.array(y_train)

8. **`word2vec`의 임베딩 가중치 행렬을 만들어줍니다.**<br/>
미리 학습된 모든 단어(300만개)에 대해 만들 경우 너무 행렬이 커지므로<br/>
vocab에 속하는 단어에 대해서만 만들어지도록 합니다. 

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))

print(np.shape(embedding_matrix))

(19999, 300)


In [ ]:
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

9. 신경망을 구성하기 위한 keras 모듈을 불러온 후<br/>
학습을 수행해줍니다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

아래 식에서 **`GlobalAveragePooling1D`** 은 처음 보는 Layer일 것입니다.<br/>
일단은 입력되는 행렬의 평균을 구하는 층, 즉 입력되는 단어 벡터의 평균을 구하는 층으로 알아두시면 되겠습니다.

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(GlobalAveragePooling1D()) # 입력되는 단어 벡터의 평균을 구하는 함수입니다.
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
313/313 [==============================] - 8s 24ms/step - loss: 0.6924 - acc: 0.5268 - val_loss: 0.6907 - val_acc: 0.5996
Epoch 2/20
313/313 [==============================] - 6s 20ms/step - loss: 0.6901 - acc: 0.5767 - val_loss: 0.6882 - val_acc: 0.5972
Epoch 3/20
313/313 [==============================] - 6s 20ms/step - loss: 0.6880 - acc: 0.5899 - val_loss: 0.6859 - val_acc: 0.6006
Epoch 4/20
313/313 [==============================] - 7s 23ms/step - loss: 0.6861 - acc: 0.5918 - val_loss: 0.6837 - val_acc: 0.5928
Epoch 5/20
313/313 [==============================] - 7s 23ms/step - loss: 0.6845 - acc: 0.5949 - val_loss: 0.6824 - val_acc: 0.5970
Epoch 6/20
313/313 [==============================] - 6s 20ms/step - loss: 0.6827 - acc: 0.5957 - val_loss: 0.6800 - val_acc: 0.5946
Epoch 7/20
313/313 [==============================] - 7s 22ms/step - loss: 0.6813 - acc: 0.6028 - val_loss: 0.6787 - val_acc: 0.6112
Epoch 8/20
313/313 [==============================] - 6s 20ms/step - 

In [ ]:
test_sentences = [decode_review(idx) for idx in X_test]

X_test_encoded = tokenizer.texts_to_sequences(test_sentences)

X_test=pad_sequences(X_test_encoded, maxlen=400, padding='post')
y_test=np.array(y_test)

In [ ]:
model.evaluate(X_test, y_test)

782/782 [==============================] - 7s 8ms/step - loss: 0.6680 - acc: 0.6093


[0.6679654121398926, 0.609279990196228]

## 🧐  Review

- 단어의 분산 표현(Distributed Representation)
    - 원-핫 인코딩(One-hot Encoding)
    - 임베딩(Embedding)

- Word2Vec
    - CBoW와 Skip-gram
    - Word2Vec의 구조
    - Word2Vec의 임베딩 벡터를 시각화한 결과

- fastText
    - OOV(Out of Vocabulary) 문제
    - 철자(Character) 단위 임베딩

### 🔝 References

- [n-gram](https://www.youtube.com/watch?v=4f9XC8HHluE) 이란 무엇일까요?
- [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759)